In [1]:
from kubeflow import fairing
from kubeflow.fairing.kubernetes import utils as k8s_utils

PRIVATE_REGISTRY = 'registry.kube-system.svc.cluster.local:30000'

fairing.config.set_preprocessor(
    'python', 
     # command = ['python'],  # default: python
     input_files = ['00-python-file-to-fairing.py'], # 생성되는 image의 /app/ 디렉토리로 복사됨
)

fairing.config.set_builder(
    'append',
    base_image = 'tensorflow/tensorflow',
    registry = PRIVATE_REGISTRY,
    image_name='my-02-python-file-fairing', 
    push=True
)

fairing.config.set_deployer(
    'job',
    # namespace='myspace', # default: 현재 네임스페이스
    pod_spec_mutators=[
        k8s_utils.get_resource_mutator(cpu=1, memory=5)]
)

fairing.config.run()

[I 210314 12:04:16 config:134] Using preprocessor: <kubeflow.fairing.preprocessors.base.BasePreProcessor object at 0x7f760de6b630>
[I 210314 12:04:16 config:136] Using builder: <kubeflow.fairing.builders.append.append.AppendBuilder object at 0x7f760de6b6d8>
[I 210314 12:04:16 config:138] Using deployer: <kubeflow.fairing.deployers.job.job.Job object at 0x7f760de6b668>
[W 210314 12:04:16 append:50] Building image using Append builder...
[I 210314 12:04:16 base:107] Creating docker context: /tmp/fairing_context_thh82pin
[I 210314 12:04:16 docker_creds_:234] Loading Docker credentials for repository 'tensorflow/tensorflow'
[W 210314 12:04:17 append:54] Image successfully built in 0.5099206780014356s.
[W 210314 12:04:17 append:94] Pushing image registry.kube-system.svc.cluster.local:30000/my-02-python-file-fairing:90474E8B...
[I 210314 12:04:17 docker_creds_:234] Loading Docker credentials for repository 'registry.kube-system.svc.cluster.local:30000/my-02-python-file-fairing:90474E8B'
[W 2

2021-03-14 12:04:18.995557: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-03-14 12:04:18.995621: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
11493376/11490434 [==============================] - 0s 0us/step
2021-03-14 12:04:21.863121: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-03-14 12:04:21.863476: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-03-14 12:04:21.863507: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-03-14 12:04:21.863534: I tensorflow/stream_executor/cuda/cuda

[W 210314 12:04:57 job:173] Cleaning up job fairing-job-ph9st...


(<kubeflow.fairing.preprocessors.base.BasePreProcessor at 0x7f760de6b630>,
 <kubeflow.fairing.deployers.job.job.Job at 0x7f760de6b668>)

# Image 확인
- Host에서 실행
- fairing에서 난수로 생성한 tag를 변경해서 실행할 것 (예: E2683407)

### Image pull
```sh
docker pull registry.kube-system.svc.cluster.local:30000/my-02-python-file-fairing:E2683407
```

### Image inspect
```sh
docker inspect registry.kube-system.svc.cluster.local:30000/my-02-python-file-fairing:E2683407
```
Cmd 항목을 확인하고 faring 코드대로 image build 되었음을 확인
```json
...
            "Cmd": [
                "python",
                "/app/00-python-file-to-fairing.py"
            ],
...
```